In [22]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import sys
import csv
import functools
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

csv.field_size_limit(1000000000000000000)

1000000000000000000

In [23]:
# import FOITEXT and MASDEV files
left = pd.read_pickle("./3_FOITEXT_File.pkl")
right = pd.read_pickle("./4_MASDEV_Censored.pkl")

In [24]:
print("shape of FOITEXT: " + str(left.shape))
print("shape of MASTER_DEVICE: " + str(right.shape))

shape of FOITEXT: (10419137, 7)
shape of MASTER_DEVICE: (26726, 41)


In [25]:
df = pd.merge(left, right, on="MDR_REPORT_KEY", how="inner", indicator=True)
df["_merge"].value_counts()

both          25671
left_only         0
right_only        0
Name: _merge, dtype: int64

In [26]:
df.head()

,MDR_REPORT_KEY,MDR_TEXT_KEY,TEXT_TYPE_CODE,PATIENT_SEQUENCE_NUMBER,DATE_REPORT,FOI_TEXT,textLen,REPORT_NUMBER,REPORT_SOURCE_CODE,MANUFACTURER_LINK_FLAG_,...,MEDTRONIC_EVERA,MEDTRONIC_VISIA,BOS_SCI_EMBLEM,StJUDE_ELLIPSE,MEDTRONIC_SPRINT,MEDTRONIC_SPRINT_QUATTRO,MEDTRONIC_SUBQ,BOS_SCI_ENDOTAK_ENDURANCE,CENSORED,_merge
0,263098,147618,D,1,NaN,OVERSENSING FROM LEAD FRACTURE CAUSED PT TO RE...,NaN,2182208-2000-00025,M,Y,...,False,False,False,False,True,False,False,False,2000-01-10,both
1,291837,148901,D,1,NaN,"SYSTEM RETURNED FOR LEAD FRACTURES, BURN MARK ...",NaN,2649622-2000-00207,M,Y,...,False,False,False,False,True,False,False,False,2000-05-12,both
2,292539,182872,D,1,NaN,PACING THRESHOLD INCREASE,NaN,2649622-2000-00385,M,Y,...,False,False,False,False,True,False,False,False,2000-07-11,both
3,292547,182873,D,1,NaN,"SENSING DIFFICULTY, INSULATION DAMAGE",NaN,2182208-2000-00251,M,Y,...,False,False,False,False,True,False,False,False,2000-07-11,both
4,292558,20930497,D,1,NaN,"APPARENT LEAD FRACTURE, NOISE.",NaN,2649622-2000-00364,M,Y,...,False,False,False,False,True,False,False,False,2000-07-11,both


In [27]:
# make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.
def cleanTextRound1(text):
    text = text.upper()
    text = re.sub('\[.*?\‘’“”…]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\n', '', text)
    return text

round1 = lambda x: cleanTextRound1(x)


data_clean = pd.DataFrame(df.FOI_TEXT.apply(round1))
data_clean.head(5)

,FOI_TEXT
0,OVERSENSING FROM LEAD FRACTURE CAUSED PT TO RE...
1,SYSTEM RETURNED FOR LEAD FRACTURES BURN MARK O...
2,PACING THRESHOLD INCREASE
3,SENSING DIFFICULTY INSULATION DAMAGE
4,APPARENT LEAD FRACTURE NOISE


In [28]:
# checking the first 100 reports in the df
n=0

for entry in data_clean["FOI_TEXT"]:
    if n < 100:
        print("[" + str(n) + "]\n" + entry + "\n")
        n += 1    

[0]
OVERSENSING FROM LEAD FRACTURE CAUSED PT TO RECEIVE UNNECESSARY ICD DISCHARGES

[1]
SYSTEM RETURNED FOR LEAD FRACTURES BURN MARK ON   LONG CHARGE TIME NOT ERI TACHY    TS    POR AFTER  SHOCKS REMAINS IMPLANTED   SECOND REPORT LCT   PTS DAUGHTER ALLEGES SHOCKS WHEN PT MOVES ARM  SYSTEM RETURNED  FOR LEAD FRACTURES BURN MARK ON  AND POWER ON RESET PARAMETERS

[2]
PACING THRESHOLD INCREASE

[3]
SENSING DIFFICULTY INSULATION DAMAGE

[4]
APPARENT LEAD FRACTURE NOISE

[5]
THRESHOLD HIGHUNSTABLEUNMEASURABLE

[6]
NO CAPTURE

[7]
HV    PROGRAMMED ONLY  DELIVERED  TS 

[8]
INAPPROPRIATE SHOCKS DURING SR DUE TO TWAVE OVERSENSING

[9]
RECEIVED WITH INSUFFICIENT INFORMATION AND TESTED OUT OF SPECIFICATIONS DURING MFGRS ANALYSIS

[10]
UNDEFINED RESISTANCE

[11]
UNWANTEDUNEXPECTED SHOCK SENSING DIFFICULTY

[12]
OVERSENSING IMPEDANCE  CHARGE CIRCUIT PROBLEM TACHY  LEAD FRACTURED ICD INACTIVE CHARGE CIRCUIT TIME OUT

[13]
NOT SENSING HIGH RESISTANCEIMPEDANCE NOT PACING APPARENT LEAD FRACTURE

[14]


In [29]:
# replace manufacturer identifiers
# spaces were not used in regexs as words may not be spaced
def cleanTextRound2(text):
    text = re.sub('''SJM|TELECTRONICS PACING SYSTEMS INC|RICE CREEK|MPROC|MDT|EOC HEERLEN DISTRIBUTION CTR
                        |BOSTON SCIENTIFIC|GUIDANT|CPI|CARDIAC PACEMAKER|CLONMEL|CAMERON HEALTH|HISTORICAL PUERTO RICO
                        |EXTERNAL MANUFACTURERPCI PLANT|SORI|ANGEION|MPRI|IPG MFG|ST JUDE|ST JUDE MEDICAL|MEDTRONIC 
                        |BOSTON SCIENTIFIC|BIOTRONIK|SORIN''', " MFGID ", text)
    return text

round2 = lambda x: cleanTextRound2(x)


data_clean = pd.DataFrame(data_clean.FOI_TEXT.apply(round2))
data_clean.head(5)

,FOI_TEXT
0,OVERSENSING FROM LEAD FRACTURE CAUSED PT TO RE...
1,SYSTEM RETURNED FOR LEAD FRACTURES BURN MARK O...
2,PACING THRESHOLD INCREASE
3,SENSING DIFFICULTY INSULATION DAMAGE
4,APPARENT LEAD FRACTURE NOISE


In [30]:
# creating a list of device identifiers
listDevices = []



for n in df["STD_DEVICE"].value_counts().index:
    listDevices.append(n)

print(listDevices)

['MEDTRONIC_EVERA', 'MEDTRONIC_SPRINT_QUATTRO', 'StJUDE_ELLIPSE', 'MEDTRONIC_SPRINT', 'BOS_SCI_EMBLEM', 'MEDTRONIC_VISIA', 'MEDTRONIC_SUBQ']


In [31]:
# replace device identifiers
# other versions of existing entries were added to account for typos
# spaces were not used in regexs as words may not be spaced
def cleanTextRound3(text):
    text = re.sub('''
                    EVERA|SPRINT QUATTRO|SPRINT|ELLIPSE|EMBLEM|VISIA|SUBQ|SUB Q|SUB-Q
                    ''', " DEVID ", text)
    return text

round3 = lambda x: cleanTextRound3(x)


data_clean = pd.DataFrame(data_clean.FOI_TEXT.apply(round3))
data_clean.head(5)

,FOI_TEXT
0,OVERSENSING FROM LEAD FRACTURE CAUSED PT TO RE...
1,SYSTEM RETURNED FOR LEAD FRACTURES BURN MARK O...
2,PACING THRESHOLD INCREASE
3,SENSING DIFFICULTY INSULATION DAMAGE
4,APPARENT LEAD FRACTURE NOISE


In [32]:
# replace "EVENT DESCRIPTION" found at the start of many entries
def cleanTextRound4(text):
    text = re.sub("EVENT DESCRIPTION", "", text)
    return text

round4 = lambda x: cleanTextRound4(x)


data_clean = pd.DataFrame(data_clean.FOI_TEXT.apply(round4))
data_clean.head(5)

,FOI_TEXT
0,OVERSENSING FROM LEAD FRACTURE CAUSED PT TO RE...
1,SYSTEM RETURNED FOR LEAD FRACTURES BURN MARK O...
2,PACING THRESHOLD INCREASE
3,SENSING DIFFICULTY INSULATION DAMAGE
4,APPARENT LEAD FRACTURE NOISE


In [33]:
# replace odd spacing
def cleanTextRound5(text):
    text = re.sub(" ", " ", text)
    text = re.sub("  ", " ", text)
    text = re.sub("   ", " ", text)
    text = re.sub("    ", " ", text)
    text = re.sub("     ", " ", text)
    return text

round5 = lambda x: cleanTextRound5(x)


data_clean = pd.DataFrame(data_clean.FOI_TEXT.apply(round5))
data_clean.head(5)

,FOI_TEXT
0,OVERSENSING FROM LEAD FRACTURE CAUSED PT TO RE...
1,SYSTEM RETURNED FOR LEAD FRACTURES BURN MARK O...
2,PACING THRESHOLD INCREASE
3,SENSING DIFFICULTY INSULATION DAMAGE
4,APPARENT LEAD FRACTURE NOISE


In [34]:
# remove stopwords
def cleanTextRound6(text):
    text = re.sub('''
                    I | ME | MY | MYSELF | WE | OUR | OURS | OURSELVES | YOU | YOUR | YOURS | YOURSELF 
                    | YOURSELVES | HE | HIM | HIS | HIMSELF | SHE | HER | HERS | HERSELF | IT | ITS | ITSELF 
                    | THEY | THEM | THEIR | THEIRS | THEMSELVES | WHAT | WHICH | WHO | WHOM | THIS | THAT | THESE 
                    | THOSE | AM | IS | ARE | WAS | WERE | BE | BEEN | BEING | HAVE | HAS | HAD | HAVING | DO 
                    | DOES | DID | DOING | A | AN | THE | AND | BUT | IF | OR | BECAUSE | AS | UNTIL | WHILE 
                    | OF | AT | BY | FOR | WITH | ABOUT | AGAINST | BETWEEN | INTO | THROUGH | DURING | BEFORE 
                    | AFTER | ABOVE | BELOW | TO | FROM | UP | DOWN | IN | OUT | ON | OFF | OVER | UNDER | AGAIN 
                    | FURTHER | THEN | ONCE | HERE | THERE | WHEN | WHERE | WHY | HOW | ALL | ANY | BOTH | EACH 
                    | FEW | MORE | MOST | OTHER | SOME | SUCH | NO | NOR | NOT | ONLY | OWN | SAME | SO | THAN 
                    | TOO | VERY | S | T | CAN | WILL | JUST | DON | SHOULD | NOW
                    ''', " ", text)
    return text

round6 = lambda x: cleanTextRound6(x)


data_clean = pd.DataFrame(data_clean.FOI_TEXT.apply(round6))
data_clean.head(5)

,FOI_TEXT
0,OVERSENSING LEAD FRACTURE CAUSED PT RECEIVE UN...
1,SYSTEM RETURNED LEAD FRACTURES BURN MARK LONG...
2,PACING THRESHOLD INCREASE
3,SENSING DIFFICULTY INSULATION DAMAGE
4,APPARENT LEAD FRACTURE NOISE


In [35]:
# checking the first 100 reports in the df
n=0

for entry in data_clean["FOI_TEXT"]:
    if n < 100:
        print("[" + str(n) + "]\n" + entry + "\n")
        n += 1    

[0]
OVERSENSING LEAD FRACTURE CAUSED PT RECEIVE UNNECESSARY ICD DISCHARGES

[1]
SYSTEM RETURNED LEAD FRACTURES BURN MARK  LONG CHARGE TIME ERI TACHY  TS  POR SHOCKS REMAINS IMPLANTED  SECOND REPORT LCT  PTS DAUGHTER ALLEGES SHOCKS PT MOVES ARM SYSTEM RETURNED LEAD FRACTURES BURN MARK AND POWER RESET PARAMETERS

[2]
PACING THRESHOLD INCREASE

[3]
SENSING DIFFICULTY INSULATION DAMAGE

[4]
APPARENT LEAD FRACTURE NOISE

[5]
THRESHOLD HIGHUNSTABLEUNMEASURABLE

[6]
NO CAPTURE

[7]
HV  PROGRAMMED DELIVERED TS 

[8]
INAPPROPRIATE SHOCKS SR DUE TWAVE OVERSENSING

[9]
RECEIVED INSUFFICIENT INFORMATION TESTED OF SPECIFICATIONS MFGRS ANALYSIS

[10]
UNDEFINED RESISTANCE

[11]
UNWANTEDUNEXPECTED SHOCK SENSING DIFFICULTY

[12]
OVERSENSING IMPEDANCE CHARGE CIRCUIT PROBLEM TACHY LEAD FRACTURED ICD INACTIVE CHARGE CIRCUIT TIME OUT

[13]
NOT SENSING HIGH RESISTANCEIMPEDANCE PACING APPARENT LEAD FRACTURE

[14]
SENSING DIFFICULTY INSULATION BREAK NEAR SUTURE SLEEVE LOW VENTRICULAR IMPEDANCERESISTANCE

[15]

In [36]:
# join the cleaned data with the main df
df["CLEAN_FOI_TEXT"] = data_clean
df["CLEAN_FOI_TEXT"].head()

0    OVERSENSING LEAD FRACTURE CAUSED PT RECEIVE UN...
1    SYSTEM RETURNED LEAD FRACTURES BURN MARK  LONG...
2                            PACING THRESHOLD INCREASE
3                 SENSING DIFFICULTY INSULATION DAMAGE
4                         APPARENT LEAD FRACTURE NOISE
Name: CLEAN_FOI_TEXT, dtype: object

In [37]:
# stemming
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def cleanTextRound7(text): 
    return " ".join([stemmer.stem(word) for word in text.split()])
df["CLEAN_FOI_TEXT"] = df["CLEAN_FOI_TEXT"].apply(cleanTextRound7)

In [38]:
# checking the first 100 reports in the df
n=0

for entry in df["CLEAN_FOI_TEXT"]:
    if n < 100:
        print("[" + str(n) + "]\n" + entry + "\n")
        n += 1    

[0]
oversens lead fractur caus pt receiv unnecessari icd discharg

[1]
system return lead fractur burn mark long charg time eri tachi ts por shock remain implant second report lct pt daughter alleg shock pt move arm system return lead fractur burn mark and power reset paramet

[2]
pace threshold increas

[3]
sens difficulti insul damag

[4]
appar lead fractur nois

[5]
threshold highunstableunmeasur

[6]
no captur

[7]
hv program deliv ts

[8]
inappropri shock sr due twave oversens

[9]
receiv insuffici inform test of specif mfgr analysi

[10]
undefin resist

[11]
unwantedunexpect shock sens difficulti

[12]
oversens imped charg circuit problem tachi lead fractur icd inact charg circuit time out

[13]
not sens high resistanceimped pace appar lead fractur

[14]
sens difficulti insul break near sutur sleev low ventricular impedanceresist

[15]
sens captur difficulti lead fractur inappropri shock pin slightli loos helix mechan nonrespons

[16]
sens difficulti

[17]
devic test of specif ma

In [39]:
df.to_pickle("./4_MASDEV_Cleaned.pkl")